In [1]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import os
import glob
import requests as r
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio as rio
import rioxarray as riox
import time
import xarray as xr
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from pandas import to_datetime
from rasterio.crs import CRS
from datetime import datetime, timedelta
from netrc import netrc
from pyproj import Proj
from src.hls_funcs import fetch
from src.hls_funcs.masks import mask_hls, shp2mask, bolton_mask, atsa_mask
from src.hls_funcs.indices import ndvi_func
from src.hls_funcs.smooth import smooth_xr, despike_ts_xr
import cartopy.crs as ccrs
from rasterio.plot import show
from src.hls_funcs.predict import pred_bm, pred_bm_se, pred_cov
import dask.diagnostics
import matplotlib.pyplot as plt
import gc


In [2]:
os.environ.update({'OMP_NUM_THREADS': '1',
                  'MKL_NUM_THREADS': '1',
                  'OPENBLAS_NUM_THREADS': '1'})

In [3]:
#wkDIR = '/project/cper_neon_aop/hls_nrt/'
#outDIR = '/90daydata/cper_neon_aop/hls_nrt/' + prefix

wkDIR = os.getcwd()
outDIR_base = 'data/hls_nrt/'

cluster_loc = 'local'

In [4]:
yr = 2022

#prefix = 'redtop' 
prefix = 'cper'

if prefix == 'cper': 
    aoi_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
    df_aoi = gpd.read_file(aoi_f)
    subunit_name_old = 'Past_Name_'
    subunit_name = 'Pasture'
elif prefix == 'redtop':
    from src.utils.convert import kmz_to_shp
    df_aoi = kmz_to_shp('data/ground/RedTop_Boundary.kmz', 'data/ground/')
    df_aoi = df_aoi.to_crs(epsg=32613)
    subunit_name_old = None
    subunit_name = None

outDIR = os.path.join(outDIR_base, prefix)

mod_bm = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
mod_cov = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))

/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator PLSRegression from version 0.24.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
os.chdir(wkDIR)
if cluster_loc == 'local':
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    aws=False
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 30, 'x': 30, 'time':-1}
    cluster = LocalCluster(n_workers=8, threads_per_worker=2)
    client = Client(cluster)
    display(client)
elif cluster_loc == 'coiled':
    import coiled
    aws=True
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 10, 'x': 10, 'time':-1}
    s3_cred = fetch.setup_netrc(creds=['spkearney', '1mrChamu'], aws=aws)
    coiled.create_software_environment(
    name="hls_cog_coiled",
    conda="hls_cog_coiled_env.yaml")
    cluster = coiled.Cluster(
        name="hls_cog_coiled",
        software="kearney-sp/hls_cog_coiled",
        n_workers=5,
        worker_cpu=2,
        scheduler_cpu=2,
        backend_options={"region": "us-west-2"},
        environ=dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    )
    client = Client(cluster)
    display(client)
elif cluster_loc == 'hpc':
    from dask.distributed import LocalCluster, Client
    import dask_jobqueue as jq
    import dask
    from jupyter_server import serverapp
    
    # get the server address for porting
    try:
        jupServer = [x for x in serverapp.list_running_servers()][0]
    except IndexError:
        # manually copy/paste the server address
        jupServer = {'base_url': '/node/ceres19-compute-98-eth.scinet.local/17710/'}
    print('   setting up moderate cluster on HPC...')
    aws=False
    fetch.setup_env(aws=aws)
    smth_chunks = {'y': 10, 'x': 10, 'time':-1}
    dask.config.set({'distributed.dashboard.link': jupServer['base_url'] + 'proxy/{port}/status'})
    partition='short',#'short','debug', 'mem', 'mem-low',
    num_processes = 4
    num_threads_per_processes = 2
    mem = 2.5*num_processes*num_threads_per_processes
    n_cores_per_job = num_processes*num_threads_per_processes
    clust = jq.SLURMCluster(queue=partition,
                            processes=num_processes,
                            cores=n_cores_per_job,
                            memory=str(mem)+'GB',
                            interface='ib0',
                            #interface='enp24s0f0',
                            local_directory='$TMPDIR',
                            death_timeout=30,
                            walltime='02:00:00',
                            job_extra=["--output=/dev/null","--error=/dev/null"])
    client=Client(clust)
    #Scale Cluster 
    num_jobs=20
    clust.scale(jobs=num_jobs)
    try:
        client.wait_for_workers(n_workers=num_jobs*num_processes, timeout=60)
    except dask.distributed.TimeoutError as e:
        print(str(num_jobs*num_processes) + ' workers not available. Continuing with available workers.')
        #print(e)
        pass
    display(client)

if not os.path.exists(outDIR):
    os.mkdir(outDIR)

if subunit_name_old is not None:
    df_aoi = df_aoi.rename(columns={subunit_name_old: subunit_name})
if subunit_name is not None:
    df_aoi = df_aoi.dissolve(by=subunit_name).reset_index()

start_date = str(yr - 1) + "-11-01"
end_date = str(yr + 1) + "-03-01"

# set the date range for analysis
date_rng = pd.date_range(start=start_date, end=end_date)
date_rng = date_rng[date_rng <= datetime.today()]

   setting up Local cluster...


2022-09-09 09:07:20,983 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-73d_8cfj', purging
2022-09-09 09:07:21,002 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-7dq837vk', purging
2022-09-09 09:07:21,020 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-8iyxte4s', purging
2022-09-09 09:07:21,037 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-aexrfppu', purging
2022-09-09 09:07:21,055 - distributed.diskutils - INFO - Found stale lock file and directory '/mnt/c/Users/TBGPEA-Sean/git_repos/hls_nrt/dask-worker-space/worker-i0ivwecd', purging
2022-09-09 09:07:21,075 - distributed.diskutils - INFO - Found stale lock file and directory '/

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 15.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35181,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 15.49 GiB
Comm: tcp://127.0.0.1:46581,Total threads: 2
Dashboard: http://127.0.0.1:38839/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:44223,


In [6]:
t00 = time.time()
nc_flist = list(filter(os.path.isfile, 
                    glob.glob(os.path.join(outDIR,
                              prefix + 
                              '_hls_tmp/' + 'hls_ds_' + str(yr) + '*.nc'))))
if len(nc_flist) > 0:
    nc_flist.sort(key=lambda x: os.path.getctime(x))
    nc_f = nc_flist[-1]
else:
    print('ERROR: No temp nc file found')

hls_ds = xr.open_dataset(nc_f).load()
#hls_atsa = xr.open_dataset(re.sub('hls_ds_', 'hls_atsa_', nc_f))

In [7]:
hls_ds

<xarray.Dataset>
Dimensions:      (time: 158, y: 323, x: 323)
Coordinates:
  * time         (time) datetime64[ns] 2021-11-02 2021-11-03 ... 2022-09-05
  * x            (x) float64 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y            (y) float64 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
Data variables:
    BLUE         (time, y, x) float64 5.745e+03 5.951e+03 ... 819.0 789.0
    GREEN        (time, y, x) float64 5.879e+03 6.046e+03 ... 1.105e+03
    RED          (time, y, x) float64 5.796e+03 5.968e+03 ... 1.367e+03
    NIR1         (time, y, x) float64 6.094e+03 6.236e+03 ... 2.444e+03
    SWIR1        (time, y, x) float64 3.182e+03 3.249e+03 ... 3.495e+03
    SWIR2        (time, y, x) float64 3e+03 3.047e+03 ... 2.664e+03 2.597e+03
    FMASK        (time, y, x) float64 194.0 194.0 194.0 194.0 ... 64.0 64.0 64.0
    SZA          (time, y, x) float64 5.661e+03 5.661e+03 ... 3.707e+03
    SAA          (time, y, x) float64 1.684e+04 1.684e+04 ... 1.527e+04
    VZA          (time, y, x) float64 628.0 629.0 629.0 ... 967.0 967.0 967.0
    VAA          (time, y, x) float64 2.961e+04 2.96e+04 ... 9.532e+03 9.532e+03
    maskcov_pct  (time) float64 100.0 0.01342 65.75 ... 58.58 0.009585 0.03451
Attributes:
    crs:         epsg:32613
    transform:   [ 3.00000e+01  0.00000e+00  5.17590e+05  0.00000e+00 -3.0000...
    resolution:  30

In [8]:
#hls_bolton_mask = bolton_mask(hls_ds).compute()
#hls_mask_snow = mask_hls(hls_ds['FMASK'], mask_types=['snow'])
#hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['all'])

#show((hls_bolton.sum(dim='time') / len(hls_mask.time)) > 0.1)

In [9]:
hls_atsa = atsa_mask(hls_ds).compute()

/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/home/spkearney/miniconda3/envs/hls_nrt_env/lib/py

HOT centers of clear, thin and thick cloudy pixels on land : [ 348.54946358 1731.36178926 2951.45523538]
Shadow index centers of shadow, nonshadow dark and nonshadow bright on land : [ 667.31615187 2002.87519523 5306.47236026]


In [10]:
client.restart()

2022-09-09 09:10:33,432 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:33,455 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:33,470 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:33,493 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:33,495 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:33,497 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:33,521 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:33,535 - distributed.nanny - WARNING - Restarting worker


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 15.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35181,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: 3 minutes ago,Total memory: 15.49 GiB
Comm: tcp://127.0.0.1:33043,Total threads: 2
Dashboard: http://127.0.0.1:40873/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:44223,


In [11]:
hls_ds = xr.merge([hls_ds, hls_atsa], join='inner')
#hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['all'])
hls_mask = mask_hls(hls_ds['FMASK'], mask_types=['cirrus',
                                                'cloud',
                                                'cloud_adj',
                                                'shadow', 
                                                'snow',
                                                'water'])
#hls_mask = mask_hls(hls_ds['FMASK'], 'all')
hls_ds = hls_ds.where(hls_mask == 0)

In [12]:
hls_ds = hls_ds.where(hls_ds['ATSA'] == 1)

In [13]:
t0 = time.time()
#print('   applying tertiary cloud mask...')
#hls_ds = hls_ds.chunk({'time': -1, 'y': 100, 'x': 100})
t1 = time.time()
#hls_bolton_mask = bolton_mask(hls_ds).compute()
#hls_ds = hls_ds.where(hls_bolton_mask == 0, drop=True)
# mask out scenes where < 75% of the region is cloud-free
partial_mask = (hls_ds['NIR1'].isnull().sum(dim=['y', 'x'])/np.product(hls_ds['NIR1'].shape[1:])) < 0.25
partial_mask = partial_mask#.persist()
hls_ds = hls_ds.where(partial_mask, drop=True).compute()
print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

...completed in 0.0 secs
total elasped time: 0.01 mins



In [14]:
hls_ds

<xarray.Dataset>
Dimensions:      (time: 70, y: 323, x: 323)
Coordinates:
  * time         (time) datetime64[ns] 2021-11-03 2021-11-07 ... 2022-09-05
  * x            (x) float64 5.176e+05 5.176e+05 ... 5.272e+05 5.272e+05
  * y            (y) float64 4.524e+06 4.524e+06 ... 4.515e+06 4.515e+06
Data variables: (12/13)
    BLUE         (time, y, x) float64 718.0 744.0 741.0 ... 912.0 819.0 789.0
    GREEN        (time, y, x) float64 1.053e+03 1.078e+03 ... 1.105e+03
    RED          (time, y, x) float64 1.39e+03 1.432e+03 ... 1.426e+03 1.367e+03
    NIR1         (time, y, x) float64 2.023e+03 2.132e+03 ... 2.444e+03
    SWIR1        (time, y, x) float64 2.468e+03 2.711e+03 ... 3.495e+03
    SWIR2        (time, y, x) float64 1.721e+03 1.903e+03 ... 2.597e+03
    ...           ...
    SZA          (time, y, x) float64 5.815e+03 5.815e+03 ... 3.707e+03
    SAA          (time, y, x) float64 1.614e+04 1.614e+04 ... 1.527e+04
    VZA          (time, y, x) float64 529.0 528.0 528.0 ... 967.0 967.0 967.0
    VAA          (time, y, x) float64 1.007e+04 1.007e+04 ... 9.532e+03
    maskcov_pct  (time, y, x) float64 0.01342 0.01342 ... 0.03451 0.03451
    ATSA         (time, y, x) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0
Attributes:
    crs:         epsg:32613
    transform:   [ 3.00000e+01  0.00000e+00  5.17590e+05  0.00000e+00 -3.0000...
    resolution:  30

In [15]:
client.restart()

2022-09-09 09:10:38,348 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:38,350 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:38,354 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:38,358 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:38,392 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:38,393 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:38,394 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:38,394 - distributed.nanny - WARNING - Restarting worker


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 15.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35181,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: 3 minutes ago,Total memory: 15.49 GiB
Comm: tcp://127.0.0.1:38877,Total threads: 2
Dashboard: http://127.0.0.1:35941/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:44223,


In [16]:
if not os.path.exists(os.path.join(outDIR, 'hls_ndvi/')):
    os.mkdir(os.path.join(outDIR, 'hls_ndvi/'))
if not os.path.exists(os.path.join(outDIR, 'hls_biomass/')):
    os.mkdir(os.path.join(outDIR, 'hls_biomass/'))
if not os.path.exists(os.path.join(outDIR, 'hls_cover/')):
    os.mkdir(os.path.join(outDIR, 'hls_cover/'))

In [17]:
hls_ds = hls_ds.chunk({'time': 1, 'y': -1, 'x': -1})
print('   computing NDVI for available dates...')
# create ndvi array
xr_ndvi = ndvi_func(hls_ds)#.compute()
xr_ndvi.name = 'NDVI'


print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_ndvi['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_ndvi['time'].values],
                             'x': xr_ndvi.x,
                             'y': xr_ndvi.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(NDVI=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['NDVI'], xr_ndvi], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

yr_mask = dat_out_ds['time'].astype(np.datetime64).dt.year == yr

dat_out_ds = dat_out_ds.chunk({'time': -1, 'y': 20, 'x': 20})
dat_out_da = dat_out_ds['NDVI'].interpolate_na('time').rolling(time=21, 
                                                               center=True,
                                                               min_periods=1).mean().sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()
dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_ndvi/' + prefix + '_hls_ndvi_' + str(yr) + '.nc'))

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

   computing NDVI for available dates...
   creating daily template for output...
   interpolating and saving to disk
...completed in 15.0 secs
total elasped time: 0.29 mins



2022-09-09 09:10:55,442 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:55,444 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:55,456 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:55,471 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:55,482 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:55,489 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:55,500 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:55,510 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:10:55,517 - distributed.worker_memory - WARNING - Worker exceeded 95% memory budget. Restarting
2022-09-09 09:10:55,522 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:11:43,056 - distributed.nanny - ERROR - Restart timed out after 48.0s; returning before finished
2022-09-09 09:11:43,057 - distributed.scheduler - ERROR - Not all workers responded positively: ['OK', 'timed out

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 15.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35181,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: 4 minutes ago,Total memory: 15.49 GiB
Comm: tcp://127.0.0.1:37637,Total threads: 2
Dashboard: http://127.0.0.1:45187/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:44223,


In [18]:
print('   computing biomass for available dates...')
# create biomass array
xr_bm = hls_ds.map_blocks(pred_bm, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm.name = 'Biomass'

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_bm['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_bm['time'].values],
                             'x': xr_bm.x,
                             'y': xr_bm.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(Biomass=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['Biomass'], xr_bm], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

dat_out_ds['Biomass'] = xr.concat([dat_out['Biomass'], xr_bm], dim='time')
dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})
dat_out_da = dat_out_ds['Biomass'].interpolate_na('time').rolling(time=21, 
                                                                  center=True,
                                                                  min_periods=1).mean().sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()
dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_' + str(yr) + '.nc'))

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

   computing biomass for available dates...
   creating daily template for output...
   interpolating and saving to disk
...completed in 19.0 secs
total elasped time: 1.4 mins



2022-09-09 09:12:02,608 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:02,627 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:02,646 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:02,664 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:02,669 - distributed.worker_memory - WARNING - Worker exceeded 95% memory budget. Restarting
2022-09-09 09:12:02,673 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:02,685 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:02,686 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:02,752 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:02,792 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:12:49,954 - distributed.nanny - ERROR - Restart timed out after 48.0s; returning before finished
2022-09-09 09:12:49,955 - distributed.scheduler - ERROR - Not all workers responded positively: ['OK', 'OK', 'tim

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 15.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35181,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: 5 minutes ago,Total memory: 15.49 GiB
Comm: tcp://127.0.0.1:46515,Total threads: 2
Dashboard: http://127.0.0.1:37721/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:44223,


In [19]:
print('   computing biomass SE for available dates...')
# create biomass SE array
xr_bm_se = hls_ds.map_blocks(pred_bm_se, template=hls_ds['NIR1'],
                          kwargs=dict(model=mod_bm)).where(hls_ds['NIR1'].notnull())#.compute()
xr_bm_se.name = 'Biomass_SE'

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_bm_se['time'].max().values]

# create an output Dataset template with all dates
dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_bm_se['time'].values],
                             'x': xr_bm_se.x,
                             'y': xr_bm_se.y})

print('   interpolating and saving to disk')
t1 = time.time()

dat_out_nans = np.zeros((dat_out.dims['time'], 
                         dat_out.dims['y'], 
                         dat_out.dims['x'])) * np.nan

dat_out = dat_out.assign(Biomass_SE=(['time', 'y', 'x'],
                               dat_out_nans))

dat_out_ds = xr.concat([dat_out['Biomass_SE'], xr_bm_se], dim='time').to_dataset()
dat_out_ds = dat_out_ds.sortby('time')
dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

dat_out_ds['Biomass_SE'] = xr.concat([dat_out['Biomass_SE'], xr_bm_se], dim='time')
dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})
dat_out_da = dat_out_ds['Biomass_SE'].interpolate_na('time').rolling(time=21, 
                                                                     center=True,
                                                                     min_periods=1).mean().sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()
dat_out_da.to_netcdf(
    os.path.join(outDIR, 'hls_biomass/' + prefix + '_hls_bm_se_' + str(yr) + '.nc'))

print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

del dat_out_da, dat_out_ds, dat_out, dat_out_nans
gc.collect()
client.restart()

   computing biomass SE for available dates...
   creating daily template for output...
   interpolating and saving to disk
...completed in 18.0 secs
total elasped time: 2.51 mins



2022-09-09 09:13:09,064 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:09,073 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:09,082 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:09,096 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:09,104 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:09,112 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:09,116 - distributed.worker_memory - WARNING - Worker exceeded 95% memory budget. Restarting
2022-09-09 09:13:09,121 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:09,160 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:09,169 - distributed.nanny - WARNING - Restarting worker

Traceback (most recent call last):
  File "/home/spkearney/miniconda3/envs/hls_nrt_env/lib/python3.10/site-packages/distributed/utils.py", line 759, in wrapper
    return await func(*args, **kwargs)
GeneratorExit
Task was d

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 15.49 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35181,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: 5 minutes ago,Total memory: 15.49 GiB
Comm: tcp://127.0.0.1:39313,Total threads: 2
Dashboard: http://127.0.0.1:44573/status,Memory: 1.94 GiB
Nanny: tcp://127.0.0.1:44223,


In [20]:
print('   computing cover for available dates...')
# create cover array
xr_cov = pred_cov(hls_ds, 
                  model=mod_cov).where(hls_ds['NIR1'].notnull()).compute()

print('   creating daily template for output...')
# reset date range to be within dataset
date_rng = date_rng[date_rng <= xr_cov['time'].max().values]

for c in ['BARE', 'SD', 'GREEN', 'LITT']:
    print(c)
    # create an output Dataset template with all dates
    dat_out = xr.Dataset(coords={'time': [x for x in date_rng if x not in xr_cov['time'].values],
                                 'x': xr_cov.x,
                                 'y': xr_cov.y})

    print('   interpolating and saving to disk')
    t1 = time.time()

    dat_out_nans = np.zeros((dat_out.dims['time'], 
                             dat_out.dims['y'], 
                             dat_out.dims['x'])) * np.nan

    dat_out = dat_out.assign(TMP=(['time', 'y', 'x'],
                                   dat_out_nans))
    dat_out = dat_out.rename({'TMP': c})

    dat_out_ds = xr.concat([dat_out[c], xr_cov[c]], dim='time').to_dataset()
    dat_out_ds = dat_out_ds.sortby('time')
    dat_out_ds = dat_out_ds.rio.write_crs(CRS.from_dict(init='epsg:32613'))

    dat_out_ds[c] = xr.concat([dat_out[c], xr_cov[c]], dim='time')
    dat_out_ds = dat_out_ds.sortby('time').chunk({'time': -1, 'y': 20, 'x': 20})
    dat_out_da = dat_out_ds[c].interpolate_na('time').rolling(time=21, 
                                                              center=True,
                                                              min_periods=1).mean().sel(time=yr_mask).astype('float32').rio.reproject("EPSG:3857").compute()
    dat_out_da.to_netcdf(
        os.path.join(outDIR, 'hls_cover/' + prefix + '_hls_' + c + '_' + str(yr) + '.nc'))

    print('...completed in ' + str(round(time.time() - t1, 0)) + ' secs')
    print('total elasped time: ' + str(round((time.time() - t0)/60, 2)) + ' mins\n')

    del dat_out_da, dat_out_ds, dat_out, dat_out_nans
    gc.collect()
    client.restart()

   computing cover for available dates...
   creating daily template for output...
BARE
   interpolating and saving to disk
...completed in 14.0 secs
total elasped time: 2.95 mins



2022-09-09 09:13:35,646 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:35,668 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:35,670 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:35,694 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:35,704 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:35,706 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:35,733 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:35,743 - distributed.nanny - WARNING - Restarting worker


SD
   interpolating and saving to disk
...completed in 13.0 secs
total elasped time: 3.2 mins



2022-09-09 09:13:50,543 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:50,553 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:50,566 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:50,571 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:50,581 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:50,584 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:50,599 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:13:50,632 - distributed.nanny - WARNING - Restarting worker


GREEN
   interpolating and saving to disk
...completed in 14.0 secs
total elasped time: 3.47 mins



2022-09-09 09:14:06,257 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:06,291 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:06,310 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:06,312 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:06,316 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:06,320 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:06,326 - distributed.worker_memory - WARNING - Worker exceeded 95% memory budget. Restarting
2022-09-09 09:14:06,333 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:06,356 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:06,374 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:14:53,907 - distributed.nanny - ERROR - Restart timed out after 48.0s; returning before finished
2022-09-09 09:14:53,908 - distributed.scheduler - ERROR - Not all workers responded positively: ['OK', 'timed out

LITT
   interpolating and saving to disk
...completed in 13.0 secs
total elasped time: 4.49 mins



2022-09-09 09:15:07,564 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:07,593 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:07,654 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:07,663 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:07,667 - distributed.worker_memory - WARNING - Worker exceeded 95% memory budget. Restarting
2022-09-09 09:15:07,678 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:07,680 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:07,682 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:07,683 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:07,689 - distributed.nanny - WARNING - Restarting worker
2022-09-09 09:15:55,206 - distributed.nanny - ERROR - Restart timed out after 48.0s; returning before finished
2022-09-09 09:15:55,207 - distributed.scheduler - ERROR - Not all workers responded positively: ['OK', 'timed out